In [1]:
from pathlib import Path

while Path.cwd().name != 'ambient':
    %cd ..

/mmfs1/gscratch/xlab/alisaliu/ambient/notebooks
/mmfs1/gscratch/xlab/alisaliu/ambient


In [2]:
import pandas as pd
from pathlib import Path
from utils.constants import id2label, label2id
from mturk.human_eval_utils import read_batch, clean_batch, statistics_for_worker
from mturk.utils import get_disambiguation_idxs
import numpy as np
import ast
from scipy import stats
import math
import sys
np.set_printoptions(threshold=sys.maxsize)
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from utils.utils import get_mode

In [3]:
batches_dir = Path('annotation/human_eval/batches')
dirs = [d for d in os.listdir(batches_dir) if (os.path.isdir(batches_dir / d) and d.startswith('batch_'))]
hits_per_annotator = Counter()

batch_dfs = []
for batch_dir in dirs:
    batch_df = pd.read_json(batches_dir / batch_dir / 'batch_results.jsonl', lines=True)
    batch_df['labels'] = [ast.literal_eval(row['labels']) for _, row in batch_df.iterrows()]
    batch_dfs.append(batch_df)

results_df = pd.concat(batch_dfs).reset_index(drop=True)
print(f'Number of examples annotated: {len(results_df.index)}')

Number of examples annotated: 300


In [4]:
results_df['correct'] = False
for i, row in results_df.iterrows():
    disambiguation_idxs = [j for j, label in enumerate(row['labels']) if label]
    if len(disambiguation_idxs) >= 2:
        if np.all([(get_mode(row[f'd{j+1}_gold']) == True) and (get_mode(row[f'q{j+1}_gold']) == row['labels'][j]) for j in disambiguation_idxs]):
            results_df.at[i, 'correct'] = True

In [5]:
for model, model_df in results_df.groupby('source'):
    print(model, len(model_df.index), np.round(np.sum(model_df.correct)/len(model_df.index)*100, 1))

davinci 50 2.0
flan-t5-xxl 50 0.0
gpt-3.5-turbo 50 18.0
gpt-4 50 32.0
llama-65b 50 10.0
text-davinci-003 50 4.0
